 Step 1: Simulate Monthly Income/Expenses

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

np.random.seed(42)

categories_expense = ['Rent', 'Groceries', 'Transport', 'Entertainment', 'Utilities', 'Healthcare', 'Dining Out']
categories_income = ['Salary', 'Freelance', 'Investment Return']
payment_methods = ['Credit Card', 'Debit Card', 'Cash', 'Bank Transfer']
types = ['Expense', 'Income']

dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
data = []

for date in dates:
    for _ in range(np.random.randint(1, 4)):
        tx_type = np.random.choice(types, p=[0.75, 0.25])  # more expenses than income

        if tx_type == 'Expense':
            category = np.random.choice(categories_expense)
            amount = np.round(np.random.normal(50, 30), 2)
            amount = max(5, amount)
        else:
            category = np.random.choice(categories_income)
            amount = np.round(np.random.normal(2000, 1000), 2)
            amount = max(500, amount)

        data.append({
            'Date': date,
            'Category': category,
            'Amount': amount,
            'Type': tx_type,
            'Payment Method': np.random.choice(payment_methods),
            'Notes': ''
        })

df = pd.DataFrame(data)
df.to_csv('personal_budget_data.csv', index=False)


Step 2: Load & Clean Data

In [2]:
df = pd.read_csv('personal_budget_data.csv', parse_dates=['Date'])

# Create additional columns
df['Month'] = df['Date'].dt.to_period('M').astype(str)


Step 3: Category-wise Spend Analysis

In [3]:
expenses = df[df['Type'] == 'Expense']
spend_by_category = expenses.groupby('Category')['Amount'].sum().sort_values(ascending=False).reset_index()


Plot: Total Spend by Category

In [4]:
import plotly.express as px

fig1= px.bar(
    spend_by_category,
    x='Category',
    y='Amount',
    title='Total Expenses by Category',
    color='Amount',
    text='Amount'
)
fig1.show()


Step 4: Monthly Budget Dashboard

In [5]:
monthly_summary = df.groupby(['Month', 'Type'])['Amount'].sum().reset_index()
monthly_pivot = monthly_summary.pivot(index='Month', columns='Type', values='Amount').fillna(0)
monthly_pivot['Net Savings'] = monthly_pivot['Income'] - monthly_pivot['Expense']


Plot: Monthly Income vs Expenses

In [6]:
fig2= px.line(
    monthly_pivot.reset_index(),
    x='Month',
    y=['Income', 'Expense', 'Net Savings'],
    title='Monthly Income, Expenses & Net Savings',
    markers=True
)
fig2.show()


Step 5: Forecasting Future Spend (Simple)

In [7]:
from sklearn.linear_model import LinearRegression
import numpy as np

monthly_pivot_reset = monthly_pivot.reset_index()
monthly_pivot_reset['Month_Num'] = np.arange(len(monthly_pivot_reset))

X = monthly_pivot_reset[['Month_Num']]
y = monthly_pivot_reset['Expense']

model = LinearRegression()
model.fit(X, y)

future_months = np.arange(len(X), len(X)+3).reshape(-1, 1)
predicted_expense = model.predict(future_months)


c:\Users\GC\.conda\envs\youtube_analytics\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



 Plot: Forecasted Expense

In [8]:
forecast_df = pd.DataFrame({
    'Month': [f'2025-{i+1:02}' for i in range(3)],
    'Expense': predicted_expense
})

fig3= px.line(
    pd.concat([
        monthly_pivot_reset[['Month', 'Expense']],
        forecast_df
    ]),
    x='Month',
    y='Expense',
    title='Expense Forecast (Next 3 Months)',
    markers=True
)
fig3.show()


Step 6: Payment Method Breakdown

In [9]:
pay_summary = expenses.groupby('Payment Method')['Amount'].sum().reset_index()

fig4= px.pie(
    pay_summary,
    names='Payment Method',
    values='Amount',
    title='Spending by Payment Method'
)
fig4.show()


🧠 Insights You Can Offer Clients:
Where most of the money is going (categories, methods)

How they’re tracking against income/savings goals

Expense forecast for next few months

Cash vs credit usage patterns

Exporting Plots to Single HTML file

In [11]:
import plotly.io as pio

# List of all your figures
figures = [fig1, fig2, fig3, fig4]

# Generate HTML strings for each figure
html_parts = [pio.to_html(fig, full_html=False, include_plotlyjs='cdn') for fig in figures]

# Combine them into one full HTML document
full_html = f"""
<html>
<head>
    <title>Spotify Analysis</title>
</head>
<body>
    {''.join(html_parts)}
</body>
</html>
"""

# Save to file
output_path = "Budget Tracker & Personal Finance Analysis.html"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(full_html)

print(f"Saved to {output_path}")

Saved to Budget Tracker & Personal Finance Analysis.html
